# Example 6:  Simple Scheduling for Notebooks

### Author: AWS Professional Services Emerging Technology and Intelligent Platforms Group
### Date: January 30 2019

In this example, we will demonstrate how users can simply schedule notebooks to run

In [1]:
from aws_orbit_sdk import controller
from aws_orbit_sdk.magics.orbit import OrbitWorkbenchMagics 

In [2]:
!rm -f /efs/shared/regression/notebooks/B-DataAnalyst/Example-1-SQL-Analysis-Athena/yyy*
!rm -f /efs/shared/regression/notebooks/B-DataAnalyst/Example-1-SQL-Analysis-Athena/xxx*


zsh:1: no matches found: /efs/shared/regression/notebooks/B-DataAnalyst/Example-1-SQL-Analysis-Athena/yyy*
zsh:1: no matches found: /efs/shared/regression/notebooks/B-DataAnalyst/Example-1-SQL-Analysis-Athena/xxx*


In [3]:
%%run_notebook
{    
      "tasks":  [
            {
                  "notebookName": "Example-1-SQL-Analysis-Athena.ipynb",
                  "sourcePath": "shared/samples/notebooks/B-DataAnalyst",
                  "targetPath": "shared/regression/notebooks/B-DataAnalyst",
                  "targetPrefix": "yyy",
                  "params": {
                        "glue_db" : "cms_raw_db",
                        "target_db" : "users"
                  }      
            }
      ]  
}


INFO:root:using default profile {'display_name': 'Micro', 'slug': 'micro', 'description': '2 CPU + 2G MEM', 'kubespawner_override': {'cpu_guarantee': 2, 'cpu_limit': 2, 'mem_guarantee': '2G', 'mem_limit': '2G'}, 'default': True}
INFO:root:volumes:[{"name": "efs-volume", "persistentVolumeClaim": {"claimName": "jupyterhub"}}]
INFO:root:volume_mounts:[{"mountPath": "/efs", "name": "efs-volume"}]


{'ExecutionType': 'eks',
 'Identifier': 'orbit-lake-user-fargate-runner-8ckzc',
 'NodeType': 'fargate',
 'tasks': [{'notebookName': 'Example-1-SQL-Analysis-Athena.ipynb',
   'sourcePath': 'shared/samples/notebooks/B-DataAnalyst',
   'targetPath': 'shared/regression/notebooks/B-DataAnalyst',
   'targetPrefix': 'yyy',
   'params': {'glue_db': 'cms_raw_db', 'target_db': 'users'}}]}

In [5]:
%%schedule_notebook -cron 0/2 * 1/1 * ?  -id yyy
{    
      "tasks":  [
            {
                  "notebookName": "Example-1-SQL-Analysis-Athena.ipynb",
                  "sourcePath": "shared/samples/notebooks/B-DataAnalyst",
                  "targetPath": "shared/regression/notebooks/B-DataAnalyst",
                  "targetPrefix": "yyy",
                  "params": {
                        "glue_db" : "cms_raw_db",
                        "target_db" : "users"
                  }      
            }
      ]  
}


INFO:root:using default profile {'display_name': 'Micro', 'slug': 'micro', 'description': '2 CPU + 2G MEM', 'kubespawner_override': {'cpu_guarantee': 2, 'cpu_limit': 2, 'mem_guarantee': '2G', 'mem_limit': '2G'}, 'default': True}
INFO:root:volumes:[{"name": "efs-volume", "persistentVolumeClaim": {"claimName": "jupyterhub"}}]
INFO:root:volume_mounts:[{"mountPath": "/efs", "name": "efs-volume"}]


{'ExecutionType': 'eks', 'Identifier': 'orbit-lake-user-yyy'}

In [ ]:
controller.schedule_notebooks(triggerName='xxx', frequency="0/2 * 1/1 * ?", taskConfiguration={    
      "tasks":  [
            {
                  "notebookName": "Example-1-SQL-Analysis-Athena.ipynb",
                  "sourcePath": "shared/samples/notebooks/B-DataAnalyst",
                  "targetPath": "shared/regression/notebooks/B-DataAnalyst",
                  "targetPrefix": "xxx",
                  "params": {
                        "glue_db" : "cms_raw_db",
                        "target_db" : "users"
                  }      
            }
      ]  
    }
                           
)

In [ ]:
! date

In [ ]:
%%time 

import time
found = False
for k in range(1,20):
    print(f"waiting one more min...")
    time.sleep(60)
    found = False
    executions = controller.get_execution_history("shared/regression/notebooks/B-DataAnalyst", "Example-1-SQL-Analysis-Athena.ipynb")
    for index, row in executions.iterrows():
        print(row['relativePath'])
        if 'xxx' in row['relativePath']:
            if 'error@' in row['relativePath']:
                raise AssertionError('error in ' + row['relativePath'])
            found = True
            print(f"Found {row['relativePath']}")
    if found:
        break

if not found:
    raise AssertionError('cannot find executed notebook by schedule_notebooks()')
    
print("SUCCESS")

In [ ]:
!date 

In [ ]:
%delete_schedule_task -id yyy
%delete_schedule_task -id xxx
